In [1]:
#import warnings
#warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import scipy as scp
import sklearn as skl
import sklearn.datasets
import matplotlib as mplot
import scprep
from pyckmeans import MultiCKMeans, NucleotideAlignment, pcoa, CKmeans
#from IPythonh.display import display
np.random.seed(0)

In [ ]:
# Podemos simular un conjunto de datos para ver las métricas de clustering utilizadas.
x, _ = sklearn.datasets.make_blobs(n_samples=50,n_features=2,centers=3,random_state=75)

In [ ]:
# Aplicamos consenso K-means.
consensokmeans = CKmeans(k=3, n_rep=100, p_samp=0.8, p_feat=0.5)
consensokmeans.fit(x)
conkm_rest = consensokmeans.predict(x)

In [ ]:
# Obtenemos matriz del consenso y el clustering del consenso.
conkm_rest.plot(figsize=(7,7))
conkm_rest.cmatrix

In [ ]:
# Métricas del consenso
print('Criterio de información Bayesiana', conkm_rest.bic)
print('Índice Davies Bouldin', conkm_rest.db)
print('Índice Calinski-Harabasz', conkm_rest.ch)
print('Clústeres del consenso\n', conkm_rest.cl)

In [2]:
# Cargar los datos desde archivo.
alnT1 = NucleotideAlignment.from_file('data/ct85_msl68.snps.phy')
alnT2 = NucleotideAlignment.from_file('data/datamatrix.snps.phy')
print(alnT1, alnT2)

<NucleotideAlignment; #samples: 108, #sites: 6752> <NucleotideAlignment; #samples: 10, #sites: 639>


In [3]:
# Calculamos distancia Kimura con 2 parámetros.
dist = alnT1.distance(distance_type='k2p')

In [4]:
# Aplicamos Principal Coordinates Analysis.
restValues = pcoa(dist, correction='lingoes')

In [5]:
# Obtenemos los valores propios hasta que el cúmulo corregido sea filtrado.
array = restValues.get_vectors(filter_by='eigvals_rel_corrected_cum', filter_th=0.8, out_format='pandas')

In [6]:
# Aplicamos consenso sobre múltiples K con K-means
multiconsensus = MultiCKMeans(k=range(2,20), n_rep=50, p_samp=0.8, p_feat=0.8)

In [7]:
# Entrenamos los datos y construimos una matriz binaria para obtener el clustering.
multiconsensus.fit(array)
mult_res = multiconsensus.predict(array)
mult_res.plot_metrics(figsize=(12,7))

D:\Programs\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
D:\Programs\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
D:\Programs\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
D:\Programs\anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, 

KeyboardInterrupt: 

In [ ]:
# Seleccionamos una -buena- K para nuestros resultados, esto para obtener un óptimo local.
consensuskmeans_res = mult_res.ckmeans_results[5]
consensuskmeans_res.plot(figsize=(14,14))